<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>Proyecto de clasificación binaria</font></h1>

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial se trabajará en un proyecto de clasificación binaria:
* Cómo diseñar y entrenar una red neuronal.
* Cómo evaluar el rendimiento de un modelo de red neuronal.
* Cómo realizar la preparación de datos para mejorar la habilidad al usar redes neuronales.
* Cómo optimizar la topología y configuración de redes neuronales.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Rendimiento del modelo de red neuronal de referencia</font>

En este problema vamos a utilizar un problema de clasificación binaria como es Sonar en el cual los resultados de Accuracy rondan el 84%.

Creemos un modelo de referencia y un resultado para este problema. Comenzaremos importando todas las clases y funciones que necesitaremos.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Sonar](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks))

In [2]:
# Binary Classification with Sonar Dataset: Baseline
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold

# load dataset

dataframe=pd.read_csv("sonar.csv",header=None)
dataset=dataframe.values

# split into input (X) and output (Y) variables

X=dataset[:,0:60].astype(float)
y=dataset[:,60]


In [3]:
X

array([[0.02  , 0.0371, 0.0428, ..., 0.0084, 0.009 , 0.0032],
       [0.0453, 0.0523, 0.0843, ..., 0.0049, 0.0052, 0.0044],
       [0.0262, 0.0582, 0.1099, ..., 0.0164, 0.0095, 0.0078],
       ...,
       [0.0522, 0.0437, 0.018 , ..., 0.0138, 0.0077, 0.0031],
       [0.0303, 0.0353, 0.049 , ..., 0.0079, 0.0036, 0.0048],
       [0.026 , 0.0363, 0.0136, ..., 0.0036, 0.0061, 0.0115]])

La variable de salida son de tipo string. Debemos convertirlos en valores enteros 0 y 1. Podemos hacer esto usando la clase `LabelEncoder` a través de `fit()` y `transform()`.

In [4]:
# encode class values as integers
encoder=LabelEncoder()
encoder.fit(y)
encoded_y=encoder.transform(y)
encoded_y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Para usar modelos de Keras con scikit-learn, debemos usar el contenedor `KerasClassifier`. También toma argumentos que pasará a la llamada a `fit()` como el número de épocas y el tamaño del batch. 

Comencemos por definir la función que crea nuestro modelo de línea de base. 
1. Tendrá una única capa oculta completamente conectada.
2. Se utilizara la función de activación ReLu. 
3. La capa de salida contiene una sola neurona para hacer predicciones utilizando función de activación Sigmoidal. 
4. Se usará la función de pérdida logarítmica binaria (`binary_crossentropy`). 
6. Utilizar el algoritmo de optimización Adam y Accuracy como métrica.

In [5]:
# baseline model
def create_model():
    model=Sequential()
    model.add(Dense(60,input_dim=60,activation="relu"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
    return model

In [6]:
modelo=create_model()
modelo.fit(X,encoded_y,epochs=100,batch_size=5)


Epoch 1/100
42/42 [==============================] - 1s 3ms/step - loss: 0.6915 - accuracy: 0.5192
Epoch 2/100
42/42 [==============================] - 0s 2ms/step - loss: 0.6618 - accuracy: 0.6058
Epoch 3/100
42/42 [==============================] - 0s 3ms/step - loss: 0.6412 - accuracy: 0.6731
Epoch 4/100
42/42 [==============================] - 0s 3ms/step - loss: 0.6146 - accuracy: 0.6827
Epoch 5/100
42/42 [==============================] - 0s 3ms/step - loss: 0.5923 - accuracy: 0.7115
Epoch 6/100
42/42 [==============================] - 0s 3ms/step - loss: 0.5766 - accuracy: 0.6827
Epoch 7/100
42/42 [==============================] - 0s 3ms/step - loss: 0.5541 - accuracy: 0.7308
Epoch 8/100
42/42 [==============================] - 0s 3ms/step - loss: 0.5372 - accuracy: 0.7404
Epoch 9/100
42/42 [==============================] - 0s 4ms/step - loss: 0.5152 - accuracy: 0.7500
Epoch 10/100
42/42 [==============================] - 0s 6ms/step - loss: 0.5027 - accuracy: 0.7644
Epoch 11/

42/42 [==============================] - 0s 4ms/step - loss: 0.1764 - accuracy: 0.9471
Epoch 84/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1760 - accuracy: 0.9471
Epoch 85/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1637 - accuracy: 0.9567
Epoch 86/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1652 - accuracy: 0.9567
Epoch 87/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1642 - accuracy: 0.9567
Epoch 88/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1657 - accuracy: 0.9519
Epoch 89/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1589 - accuracy: 0.9519
Epoch 90/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1596 - accuracy: 0.9519
Epoch 91/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1556 - accuracy: 0.9663
Epoch 92/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1555 - accuracy: 0.9663
Epoch 93/100


In [8]:
_,accuracy=modelo.evaluate(X,encoded_y)
print("acuracy: %.2f"%(accuracy*100))

7/7 [==============================] - 0s 4ms/step - loss: 0.1275 - accuracy: 0.9760
acuracy: 97.60


In [9]:
len(X)

208

In [10]:
#hacer prediccion
prediccion=modelo.predict(X)
len(prediccion)

7/7 [==============================] - 0s 3ms/step


208

In [11]:
prediccion

array([[6.97255671e-01],
       [9.19836879e-01],
       [8.79068196e-01],
       [9.59894657e-01],
       [9.18854773e-01],
       [9.70962048e-01],
       [9.29333687e-01],
       [5.52648783e-01],
       [6.99267805e-01],
       [6.82345629e-01],
       [9.95006025e-01],
       [9.79740739e-01],
       [6.30295515e-01],
       [9.84879375e-01],
       [8.75589907e-01],
       [9.98075008e-01],
       [8.68827462e-01],
       [6.89499736e-01],
       [9.99684453e-01],
       [5.36350906e-01],
       [8.00196052e-01],
       [9.44913328e-01],
       [8.92557144e-01],
       [9.94810343e-01],
       [9.99621987e-01],
       [9.74977136e-01],
       [3.91198605e-01],
       [8.87586534e-01],
       [7.39801109e-01],
       [8.00859332e-01],
       [9.80351686e-01],
       [9.78980780e-01],
       [9.92401302e-01],
       [6.61615133e-01],
       [9.58482385e-01],
       [7.74393260e-01],
       [9.96969342e-01],
       [9.92246985e-01],
       [9.94731486e-01],
       [9.97655332e-01],


Ahora es el momento de evaluar este modelo. Pasamos el número de épocas de entrenamiento al `KerasClassifier` y realizarmos una validación cruzada de 10-fold.

In [12]:
estimator=KerasClassifier(build_fn=create_model,epochs=100,batch_size=5,verbose=0)

kfold=StratifiedKFold(n_splits=10,shuffle=True)
result=cross_val_score(estimator,X,encoded_y,cv=kfold)
print(f"Accuracy: {float(result.mean())*100:.2f} y desvacion {float(result.std())*100:.2f}")

C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\R

Accuracy: 82.19 y desvacion 8.60


<a id="section2"></a>
# <font color="#004D7F" size=6>2. Optimizar el rendimiento con procesamiento de datos</font>

La **estandarización** preserva las distribuciones gaussianas mientras normaliza las tendencias centrales para cada atributo. Para ello utilizamos `StandardScaler` de scikit-learn. 

Es una buena práctica entrenar el procedimiento de estandarización en los datos de entrenamiento dentro de una ejecución de validación cruzada y usar la instancia de estandarización entrenada para preparar el fold de validación no etiquetada. Podemos lograr esto en scikit-learn usando una clase `Pipeline`. 


In [13]:
# Binary Classification with Sonar Dataset: Standardized
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# evaluate baseline model with standardized dataset
estimators=[]
estimators.append(("standarize",StandardScaler()))
estimators.append(("mlp",KerasClassifier(build_fn=create_model,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimators)
kfold=StratifiedKFold(n_splits=10,shuffle=True)
result=cross_val_score(pipeline,X,encoded_y,cv=kfold)
print(f"Accuracy: {float(result.mean())*100:.2f} y desvacion {float(result.std())*100:.2f}")

C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\R

Accuracy: 85.57 y desvacion 3.70


In [ ]:
estimators

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Ajuste de capas y neuronas</font>

En esta sección echamos un vistazo a dos experimentos sobre la estructura de la red: hacerla más pequeña y hacerla más grande.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Evaluar una topología más pequeña</font>

Podemos forzar un tipo de extracción de características por parte de la red restringiendo el espacio de representación en la primera capa oculta.

En este experimento, tomamos nuestro modelo de línea base con 60 neuronas en la capa oculta y lo reducimos a la mitad a 30. 

In [14]:
def create_smaller():
    model=Sequential()
    model.add(Dense(30,input_dim=60,activation="relu"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
    return model

## Results

# evaluate baseline model with standardized dataset
estimators=[]
estimators.append(("standarize",StandardScaler()))
estimators.append(("mlp",KerasClassifier(build_fn=create_smaller,epochs=100,batch_size=5,verbose=0)))

pipeline=Pipeline(estimators)
kfold=StratifiedKFold(n_splits=10,shuffle=True)
result=cross_val_score(pipeline,X,encoded_y,cv=kfold)
print(f"Accuracy: {float(result.mean())*100:.2f} y desvacion {float(result.std())*100:.2f}")

C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\R

Accuracy: 85.62 y desvacion 9.71


<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Evaluar una topología más grande</font>

Una topología de red neuronal con más capas ofrece más oportunidades para que la red extraiga características clave y las recombine de formas útiles no lineales. Nuestra red ahora tiene la topología:
```
    60 inputs -> [60 -> 30] -> 1 output
``` 

In [15]:
# larger model
def create_larger():
    model=Sequential()
    model.add(Dense(60,input_dim=60,activation="relu"))
    model.add(Dense(30,activation="relu"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
    return model

## Results
# evaluate baseline model with standardized dataset
estimators=[]
estimators.append(("standarize",StandardScaler()))
estimators.append(("mlp",KerasClassifier(build_fn=create_larger,epochs=100,batch_size=5,verbose=0)))

pipeline=Pipeline(estimators)
kfold=StratifiedKFold(n_splits=10,shuffle=True)
result=cross_val_score(pipeline,X,encoded_y,cv=kfold)
print(f"Accuracy: {float(result.mean())*100:.2f} y desvacion {float(result.std())*100:.2f}")

C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\R

Accuracy: 85.14 y desvacion 8.31


<a id="section3.3"></a>
# <font color="#004D7F" size=5>3.3. Trabajo a realizar</font>

Con un mayor ajuste de aspectos como el algoritmo de optimización y el número de épocas de entrenamiento, se espera que sean posibles más mejoras. ¿Cuál es la mejor puntuación que puede lograr en este conjunto de datos?

In [ ]:
from sklearn.model_selection import GridSearchCV

def create_model2(optimizer="rmsprop",init="glorot_uniform"):
    model=Sequential()
    model.add(Dense(60,input_dim=60,kernel_initializer=init,activation="relu"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
    return model

model=KerasClassifier(build_fn=create_model2)

# grid search epochs, batch size and optimizer
optimizers=["adam","rmsprop"]
epochs=[50,100,150]
batches=[5,10]

param_grid=dict(optimizer=optimizers,epochs=epochs,batch_size=batches)

grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=3)
grid_result=grid.fit(X,encoded_y)

# summarize results
print(f"mejor: {grid_result.best_score_} usando: {grid_result.best_params_}")

means=grid_result.cv_results_["mean_test_score"]
stds=grid_result.cv_results_["std_test_score"]
params=grid_result.cv_results_["params"]

for m,s,p in zip(means,stds,params):
    print(f"media: {m}, desviacion: {s} y parametros: {p}")